In [63]:
#Only run if needed
#!pip install pyarrow

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, year, month
from pyspark.sql.functions import col, rand
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import BucketedRandomProjectionLSH
import pyspark.sql.functions as F
import random
import os
import shutil
from pyspark.sql.functions import lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import lower, regexp_replace
from pyspark.sql.functions import col, when
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, regexp_extract
spark = SparkSession.builder \
    .appName("SplitByYearMonth") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
25/12/04 16:10:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_new = spark.read.csv(
    "file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv",
    header=True,
    inferSchema=True,
    ignoreLeadingWhiteSpace=True
)

In [4]:
df_new.show(5)

+------+--------------------+-------------+-------------------+---------+-------+----------+--------------------+--------------+----+----+----+----+----+
|origen|                date|     username|      user_fullname|n_replies|n_likes|n_retweets|                text|tweet_language| _c9|_c10|_c11|_c12|_c13|
+------+--------------------+-------------+-------------------+---------+-------+----------+--------------------+--------------+----+----+----+----+----+
|   df1|2019-05-27 11:49:...|    bitcointe|          Bitcointe|        0|      0|         0|Cardano: Digitize...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...|    3eyedbran|Bran - 3 Eyed Raven|        0|      2|         1|Another Test twee...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...|DetroitCrypto|        J. Scardina|        0|      0|         0|Current Crypto Pr...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...| mmursaleen72| Muhammad Mursaleen|        0|   

25/12/04 16:10:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv


In [5]:
df_setiment = df_new

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from functools import reduce   # <--- add this
# -----------------------
# 1. Start Spark session
# -----------------------
spark = SparkSession.builder.appName("PurePySparkSentiment").getOrCreate()

# -----------------------
# 2. Read CSV
# -----------------------
df_new = spark.read.csv(
    "file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv",
    header=True,
    inferSchema=True
)
df_setiment = df_new

# -----------------------
# 3. Clean text
# -----------------------
df_setiment = df_setiment.withColumn(
    "clean_text",
    lower(regexp_replace("text", "[^a-zA-Z0-9 ]", ""))
)

# -----------------------
# 4. Define positive / negative keywords
# -----------------------
positive_keywords = ["good", "great", "awesome", "happy", "bull", "moon", "pump", "gain"]
negative_keywords = ["bad", "terrible", "sad", "dump", "crash", "loss", "fear", "bear"]

# -----------------------
# 5. Assign label using pure PySpark
#    1 = positive, 0 = negative, 2 = neutral
# -----------------------
df_setiment = df_setiment.withColumn(
    "label",
    when(
        reduce(lambda a, b: a | b, [col("clean_text").rlike(k) for k in positive_keywords]), 1
    ).when(
        reduce(lambda a, b: a | b, [col("clean_text").rlike(k) for k in negative_keywords]), 0
    ).otherwise(2)
)

# -----------------------
# 6. Filter nulls
# -----------------------
df_setiment = df_setiment.filter(col("clean_text").isNotNull())

# -----------------------
# 7. Count labels
# -----------------------
label_counts = df_setiment.groupBy("label").count()
label_counts.show()


+-----+-----+
|label|count|
+-----+-----+
|    1|   58|
|    2|  239|
|    0|    9|
+-----+-----+



In [8]:
#Making a label column

In [9]:
df_setiment = df_setiment.filter(col("clean_text").isNotNull())

In [10]:
train, test = df_setiment.randomSplit([0.8, 0.2], seed=1234)

In [11]:
#Pipeline

In [12]:
# Tokenize the tweets
tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Convert words to term frequency vectors
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)

# Compute TF-IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")


In [13]:
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

In [14]:
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])

In [15]:
train_data, test_data = df_setiment.randomSplit([0.8, 0.2], seed=42)

In [16]:
model = pipeline.fit(train_data)

25/12/04 16:11:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv
25/12/04 16:11:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/04 16:11:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/

In [17]:
predictions = model.transform(test_data)
predictions.select("clean_text", "label", "prediction").show(10)

+--------------------+-----+----------+
|          clean_text|label|prediction|
+--------------------+-----+----------+
|bitcoin satoshi c...|    2|       2.0|
|httpstcoip0ph8uzy...|    2|       2.0|
|markdow bitcoin i...|    2|       2.0|
| could not agree ...|    2|       2.0|
|never too late to...|    2|       2.0|
|retweet and tweet...|    2|       2.0|
|them listen up  i...|    2|       2.0|
|bitcoin at 8000 i...|    2|       2.0|
|no youre really n...|    1|       2.0|
|hot hot hot      ...|    2|       2.0|
+--------------------+-----+----------+
only showing top 10 rows


25/12/04 16:11:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv


In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.851063829787234


25/12/04 16:11:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv
